In [ ]:
import json
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

def scrape_booking_hotels(url, hotels_with_pool, hotels_with_restaurant, hotels_with_parking, hotels_with_wheelchair, hotels_with_view):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    hotels = []

    hotel_cards = soup.find_all("div", class_="c82435a4b8 a178069f51 a6ae3c2b40 a18aeea94d d794b7a0f7 f53e278e95 c6710787a4")
    for card in hotel_cards:
        hotel_name = card.find("div", class_="f6431b446c a15b38c233").text.strip()
        hotel_location = card.find("span", class_="aee5343fdb def9bc142a").text.strip()
        
        # hotel score
        hotel_score_div = card.find('div', class_='a3b8729ab1')
        hotel_score = hotel_score_div.text.strip() if hotel_score_div else None 
        
        # hotel reviews
        hotel_reviews_div = card.find('div', class_='abf093bdfe f45d8e4c32 d935416c47')
        hotel_reviews = hotel_reviews_div.text.strip() if hotel_reviews_div else None     
        
        # Handle hotel price
        hotel_price = card.find('span', class_='f6431b446c fbfd7c1165 e84eb96b1f').text.strip() if card.find('span', class_='f6431b446c fbfd7c1165 e84eb96b1f').text.strip() else None
        
        beds_info = card.find('h4', class_='abf093bdfe e8f7c070a7').text.strip() if card.find('div', class_='abf093bdfe') else None
        rating_div = card.find("div", {"data-testid": "rating-stars", "aria-hidden": "true"})
        rating_span_count = len(rating_div.find_all("span", class_="fcd9eec8fb d31eda6efc c25361c37f")) if rating_div else None

        # Check if the hotel is in the list of hotels with facilities
        pool = 'yes' if hotel_name in hotels_with_pool else 'no'
        restaurant = 'yes' if hotel_name in hotels_with_restaurant else 'no'
        parking = 'yes' if hotel_name in hotels_with_parking else 'no'
        wheelchair = 'yes' if hotel_name in hotels_with_wheelchair else 'no'
        view = 'yes' if hotel_name in hotels_with_view else 'no'


        hotels.append({"Hotel Name": hotel_name, "Address": hotel_location, "Score": hotel_score, "Number of reviews": hotel_reviews, "Price": hotel_price, "Room type": beds_info, "Star rating": rating_span_count, "Room view": view, "Swimming pool": pool, "Restaurant": restaurant, "Parking": parking, "Wheelchair accessibility": wheelchair})

    return hotels

def scrape_and_save_hotels(search_url, hotels_with_pool, hotels_with_restaurant, hotels_with_parking, hotels_with_wheelchair, hotels_with_view):
    all_hotels = []

    try:
        for i in range(1, 28):
            hotels_on_page = scrape_booking_hotels(search_url + "&offset=" + str(i * 25), hotels_with_pool, hotels_with_restaurant, hotels_with_parking, hotels_with_wheelchair, hotels_with_view)
            all_hotels.extend(hotels_on_page)
    except KeyboardInterrupt:
        print("Execution interrupted by user.")

    return all_hotels

def save_to_excel(data, file_path):
    df = pd.DataFrame(data)
    df.to_excel(file_path, index=False)
    print("Data saved to:", file_path)

if __name__ == "__main__":
    # Load hotels with swimming pool from the external Excel file
    hotels_with_pool_df = pd.read_excel('hotels_with_pool.xlsx')
    hotels_with_pool = hotels_with_pool_df['names'].tolist()

    # Load hotels with Restaurant from the external Excel file
    hotels_with_restaurant_df = pd.read_excel('hotels_with_restaurant.xlsx')
    hotels_with_restaurant = hotels_with_restaurant_df['names'].tolist()
 
    # Load hotels with parking from the external Excel file
    hotels_with_parking_df = pd.read_excel('hotels_with_parking.xlsx')
    hotels_with_parking = hotels_with_parking_df['names'].tolist()
    
    # Load hotels with wheel chair from the external Excel file
    hotels_with_wheelchair_df = pd.read_excel('hotels_with_wheelchair.xlsx')
    hotels_with_wheelchair = hotels_with_wheelchair_df['names'].tolist()  
    
    # Load hotels with view from the external Excel file
    hotels_with_view_df = pd.read_excel('hotels_with_view.xlsx')
    hotels_with_view = hotels_with_view_df['names'].tolist()   
    
    search_url = 'https://www.booking.com/searchresults.en-gb.html?label=en-sa-booking-desktop-UmcGSHJHH5d7CxRJ6pHCYgS652796015661%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1012088%3Ali%3Adec%3Adm&sid=d7c87de715469e394879757ca2dd2e2d&aid=2311236&ss=Saudi+Arabia&ssne=Saudi+Arabia&ssne_untouched=Saudi+Arabia&efdco=1&lang=en-gb&src=searchresults&dest_id=186&dest_type=country&checkin=2024-06-03&checkout=2024-06-08&ltfd=1%3A5%3A5-2024_6-2024_7-2024%3A1%3A&group_adults=2&no_rooms=1&group_children=0&nflt=ht_id%3D204'
    
    hotels_data = scrape_and_save_hotels(search_url, hotels_with_pool, hotels_with_restaurant, hotels_with_parking, hotels_with_wheelchair, hotels_with_view)

    excel_file_path = "Saudi_Arabia_hotels.xlsx"
    save_to_excel(hotels_data, excel_file_path)
    


In [13]:
    # To check if hotel is relevant with pools
    KSA_hotels = pd.read_excel('Saudi_Arabia_hotels.xlsx')
    pool_hotel = pd.read_excel('hotels_with_pool.xlsx')

    hotels = ['Hotel Name', 'Score', 'Star rating']
    
    pool_hotel.rename(columns={'names': 'Hotel Name'}, inplace=True)

    pool_attributes_hotel_ratings = KSA_hotels[hotels]
    hotel_pool = pd.merge(pool_attributes_hotel_ratings, pool_hotel, on='Hotel Name', how='inner')

    hotel_pool = hotel_pool.drop_duplicates()
    hotel_pool.to_excel('hotel_pool.xlsx', index=False)
    print(merged_data)


                                     Hotel Name          Score  Star rating
0                             Our Habitas AlUla  9.3Scored 9.3          5.0
84           Bay La Sun Hotel and Marina - KAEC  8.2Scored 8.2          5.0
154                       Andalus Habitat Hotel  8.2Scored 8.2          4.0
196                           Banyan Tree AlUla  9.0Scored 9.0          5.0
207   Best Western Plus Al Qurayyat City Center  8.7Scored 8.7          4.0
...                                         ...            ...          ...
1960                      Spectrums Sari Jeddah  8.0Scored 8.0          4.0
1961   InterContinental Al Khobar, an IHG Hotel  8.0Scored 8.0          5.0
1962                              Emerald Hotel  8.2Scored 8.2          4.0
1963                Spectrums Al Salamah Jeddah  8.1Scored 8.1          4.0
1965                            Boudl Al Shatea  8.1Scored 8.1          NaN

[83 rows x 3 columns]


In [19]:
    # To check if hotel is relevant with restaurants
    KSA_hotels = pd.read_excel('Saudi_Arabia_hotels.xlsx')
    restaurant_hotel = pd.read_excel('hotels_with_restaurant.xlsx')

    hotels = ['Hotel Name', 'Score', 'Star rating']
    
    restaurant_hotel.rename(columns={'names': 'Hotel Name'}, inplace=True)

    restaurant_hotel_ratings = KSA_hotels[hotels]
    restaurant_hotel = pd.merge(restaurant_hotel_ratings, restaurant_hotel, on='Hotel Name', how='inner')

    restaurant_hotel = restaurant_hotel.drop_duplicates()
    restaurant_hotel.to_excel('restaurant_hotel.xlsx', index=False)
    print(restaurant_hotel)


                                     Hotel Name          Score  Star rating
0                             Our Habitas AlUla  9.3Scored 9.3          5.0
72       Al Farhan Hotel & Suites Hafr Al Batin  8.1Scored 8.1          3.0
144          Bay La Sun Hotel and Marina - KAEC  8.2Scored 8.2          5.0
204                           Banyan Tree AlUla  9.0Scored 9.0          5.0
212   Best Western Plus Al Qurayyat City Center  8.7Scored 8.7          4.0
...                                         ...            ...          ...
2023                   LeChateau Boutique Hotel  7.6Scored 7.6          NaN
2024                              Emerald Hotel  8.2Scored 8.2          4.0
2025                Spectrums Al Salamah Jeddah  8.1Scored 8.1          4.0
2027                           Manazil Al Dhayf  7.9Scored 7.9          NaN
2028                  Warwick Riyadh Al Wezarat  8.8Scored 8.8          4.0

[81 rows x 3 columns]


In [21]:
    # To check if hotel is relevant with parking
    KSA_hotels = pd.read_excel('Saudi_Arabia_hotels.xlsx')
    parking_hotel = pd.read_excel('hotels_with_parking.xlsx')

    hotels = ['Hotel Name', 'Score', 'Star rating']
    
    parking_hotel.rename(columns={'names': 'Hotel Name'}, inplace=True)

    parking_hotel_ratings = KSA_hotels[hotels]
    parking_hotel = pd.merge(parking_hotel_ratings, parking_hotel, on='Hotel Name', how='inner')

    parking_hotel = parking_hotel.drop_duplicates()
    parking_hotel.to_excel('parking_hotel.xlsx', index=False)
    print(parking_hotel)

                                             Hotel Name          Score  \
0                Al Farhan Hotel & Suites Hafr Al Batin  8.1Scored 8.1   
42                   Bay La Sun Hotel and Marina - KAEC  8.2Scored 8.2   
72                                Andalus Habitat Hotel  8.2Scored 8.2   
81                 همم للوحدات السكنية - الرحيلي Jeddah  8.0Scored 8.0   
207                                        Karims Hotel  8.0Scored 8.0   
...                                                 ...            ...   
2847                                       فندق المسافر  8.0Scored 8.0   
2850                       Narcissus Riyadh Hotel & Spa  8.6Scored 8.6   
2851  فندق ميريا الزهراء - حائز على جائزة أفضل الفنا...  8.6Scored 8.6   
2852                       Movenpick Makkah Hajar Tower  8.0Scored 8.0   
2853                                        Karan Hotel  7.1Scored 7.1   

      Star rating  
0             3.0  
42            5.0  
72            4.0  
81            NaN  
207        

In [22]:
    # To check if hotel is relevant with wheelchair
    KSA_hotels = pd.read_excel('Saudi_Arabia_hotels.xlsx')
    wheelchair_hotel = pd.read_excel('hotels_with_wheelchair.xlsx')

    hotels = ['Hotel Name', 'Score', 'Star rating']
    
    wheelchair_hotel.rename(columns={'names': 'Hotel Name'}, inplace=True)

    wheelchair_hotel_ratings = KSA_hotels[hotels]
    wheelchair_hotel = pd.merge(wheelchair_hotel_ratings, wheelchair_hotel, on='Hotel Name', how='inner')

    wheelchair_hotel = wheelchair_hotel.drop_duplicates()
    wheelchair_hotel.to_excel('wheelchair_hotel.xlsx', index=False)
    print(wheelchair_hotel)

                                 Hotel Name          Score  Star rating
0        Bay La Sun Hotel and Marina - KAEC  8.2Scored 8.2          5.0
80                    Andalus Habitat Hotel  8.2Scored 8.2          4.0
125                            Karims Hotel  8.0Scored 8.0          2.0
205                 Luxury hotel apartments  8.4Scored 8.4          4.0
335   Golden Bujari Bisha جولدن بوجاري بيشة  8.6Scored 8.6          4.0
...                                     ...            ...          ...
2746                            Yara Suites  7.7Scored 7.7          NaN
2747      Al Quba Al Thahbia Hotel Suites 2  5.9Scored 5.9          NaN
2748                            Boudl Heraa  7.5Scored 7.5          NaN
2749          شقق درر رامه للشقق المخدومة 4  6.1Scored 6.1          NaN
2750                Raoum Inn Hafr Al Baten  8.0Scored 8.0          NaN

[72 rows x 3 columns]


In [23]:
    # To check if hotel is relevant with view
    KSA_hotels = pd.read_excel('Saudi_Arabia_hotels.xlsx')
    view_hotel = pd.read_excel('hotels_with_view.xlsx')

    hotels = ['Hotel Name', 'Score', 'Star rating']
    
    view_hotel.rename(columns={'names': 'Hotel Name'}, inplace=True)

    view_hotel_ratings = KSA_hotels[hotels]
    view_hotel = pd.merge(view_hotel_ratings, view_hotel, on='Hotel Name', how='inner')

    view_hotel = view_hotel.drop_duplicates()
    view_hotel.to_excel('view_hotel.xlsx', index=False)
    print(view_hotel)

                                     Hotel Name          Score  Star rating
0        Al Farhan Hotel & Suites Hafr Al Batin  8.1Scored 8.1          3.0
78           Bay La Sun Hotel and Marina - KAEC  8.2Scored 8.2          5.0
143                                Karims Hotel  8.0Scored 8.0          2.0
208                           Banyan Tree AlUla  9.0Scored 9.0          5.0
216   Best Western Plus Al Qurayyat City Center  8.7Scored 8.7          4.0
...                                         ...            ...          ...
2127        Vittori Palace Hotel and Residences  8.2Scored 8.2          5.0
2128                        Swiss In Park Tabuk  8.8Scored 8.8          3.0
2129          Al Quba Al Thahbia Hotel Suites 2  5.9Scored 5.9          NaN
2130                                Boudl Heraa  7.5Scored 7.5          NaN
2131                    Raoum Inn Hafr Al Baten  8.0Scored 8.0          NaN

[100 rows x 3 columns]
